In [4]:
import warnings
warnings.filterwarnings('ignore')

In [59]:
import os
from dotenv import load_dotenv
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [26]:
from crewai import LLM
gemini_llm = LLM(
    model="gemini/gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0,
)

In [6]:
from crewai.tools.structured_tool import CrewStructuredTool
from pydantic import BaseModel
import requests

class APICallInput(BaseModel):
    data: dict

# Wrapper function to execute the API call
def tool_wrapper(*args, **kwargs):
    url = "https://solana-devnet.g.alchemy.com/v2/jK9f7FhUJarWMR9nozOhvCKB1Qdj3VYS"
    
    try:
        response = requests.post(url, json=kwargs["data"])

        if response.status_code == 200:
            return response.json()
        else:
            print('Error:', response.status_code)
            return None
    except requests.exceptions.RequestException as e:
        print('Error:', e)
        return None

# Create and return the structured tool
def create_structured_tool():
    return CrewStructuredTool.from_function(
        name='Wrapper API',
        description="A tool to wrap API calls with structured input.",
        args_schema=APICallInput,
        func=tool_wrapper,
    )

# Example usage
structured_tool = create_structured_tool()

# Execute the tool with structured input
result = structured_tool._run(**{
    "data": {"jsonrpc":"2.0", "id":1, "method":"getAccountInfo", "params":["TokenzQdBNbLqP5VEhdkAS6EPFLC1PHnBqCXEpPxuEb", {"encoding":"base64"}]}
})
print(result)

{'jsonrpc': '2.0', 'result': {'context': {'apiVersion': '2.1.8', 'slot': 355795616}, 'value': {'data': ['AgAAAL4zkcMPRYarxgednZ3r504XyexzY/JuY/2bbqm+blYz', 'base64'], 'executable': True, 'lamports': 1141440, 'owner': 'BPFLoaderUpgradeab1e11111111111111111111111', 'rentEpoch': 18446744073709551615, 'space': 36}}, 'id': 1}


In [78]:
from crewai import Agent, Task, Crew, Process, LLM
class FirstAgentOutput(BaseModel):
    task: str
    data: dict

task_define = Agent(
    role='Task Definer and Populator',
    goal='Act as a conversational agent. Process the user input and tell which task is needed to be performed after identifying if the input provided is an address, signature, amount, programId or a message. Also populate the field in curly brackets in params of the correct task in {task_params_map}.'
        'If the user is just conversing and there is no task to be performed, answer the user like a human and dont mention that this is a conversational query, just answer the user'
         'In case of an input error, clearly define the error and output it'
         'Take into consideration {memory} while answering as well',
    backstory='You are an expert conversation agent capable of retaining memory'
    'Have knowledge about all the tasks that can be requested by the user'
    'In case of airdrop task, replace the amount in the amount field as integer'
    "Able to identify the right task needed to be performed given the user query.",
    verbose=True,
    memory = True,
    llm = gemini_llm
    
)

get_task = Task(
    description='Identify the task needed to be performed in {text}. Return getAccountInfo is the user wants to send SOL to someone.'
                'If the user is just conversing and there is no task to be performed, converse with the user like a human and take into consideration {memory} as well ',
    expected_output='Give me the name of the task from getAccountInfo, getContractMetadata, getGasPrices, getTransactionDetails, getHealth. Also give the task data from the task_params_map with the populated curly braces if there are any'
                    'Give output with "task" field and "data" field if user is not conversing'
                    'Converse with the user if the user is just conversing and answer him without any additional commentary. Answer like a human. Make no mention that it is a conversational query',
    agent=task_define,
    output_json = FirstAgentOutput
)


LLM value is already an LLM object


In [ ]:
getAgent = Agent(
    role='Blockchain Information Retreiver',
    goal='Provide accurate blockchain details using the given information. You will use the output provided by the Task Definer and Populator to perform the correct task'
         'Pass the data field from the output of Task Definer and Populator to perform the task'
         'Only perform one task'
         "If asked to convert balance to SOL, divine lamports by 1000000000. Give accurate output"
          'Give outputs in natural language and not json. Use the fields in json output to convert it to natural language',
    backstory='An expert on retreiving information on blockchains',
    tools=[structured_tool],
    verbose=True,
    memory = True,
    llm = gemini_llm
    
)

LLM value is already an LLM object


In [83]:
task_params_map = {
    "getAccountInfo" : {
        "data": {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getAccountInfo",
            "params": [
                "{address}",
                {"encoding": "base64"}
            ]
        }
    },
    "getContractMetadata": {
        "data" : {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getProgramAccounts",
            "params": [
              "{address}",
              {
                "filters": [
                  {
                    "dataSize": 17
                  },
                  {
                    "memcmp": {
                      "offset": 4,
                      "bytes": "3Mc6vR"
                    }
                  }
                ]
              }
            ]
          }
    },
    "getGasPrices": {
        "data" : {
          "id":1,
          "jsonrpc":"2.0",
          "method":"getFeeForMessage",
          "params":[
            "{message}",
            {
              "commitment":"processed"
            }
          ]
        }
    },
    
    "getTransactionDetails": {
        "data" : {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTransaction",
            "params": [
                "{signature}",
                {"encoding": "jsonParsed","maxSupportedTransactionVersion":0}
            ]
        }
    },

    "getHealth": {
        "data" : {
            "jsonrpc":"2.0",
            "id":1,
            "method":"getHealth"
        }
    },

    "requestAirdrop" : {
        "data" : {
            "jsonrpc": "2.0", "id": 1,
            "method": "requestAirdrop",
            "params": [
              "{address}",
              "{amount}"
            ]
          }
    },

    "getBalance" : {
        "data" : {
            "jsonrpc": "2.0", "id": 1,
            "method": "getBalance",
            "params": [
              "{address}"
            ]
        }
    },

    "getInflationRate": {
        "data" : {
            "jsonrpc":"2.0",
            "id":1,
            "method":"getInflationRate"
        }
    },

    "getSupply" : {
        "data" : {
            "jsonrpc":"2.0",
            "id":1,
            "method":"getSupply"
        }
    },

    "getTokenAccountBalance" : {
        "data" : {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTokenAccountBalance",
            "params": [
              "{tokenAccountPubkey}"
            ]
        }
    },

    "getTokenAccountsByDelegate" : {
        "data" : {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTokenAccountsByDelegate",
            "params": [
              "{address}",
              {
                "programId": "{programId}"
              },
              {
                "encoding": "jsonParsed"
              }
            ]
        }
    },

    "getTokenAccountsByOwner" : {
        "data" : {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTokenAccountsByOwner",
            "params": [
              "{address}",
              {
                "programId": "{programId}"
              },
              {
                "encoding": "jsonParsed"
              }
            ]
        }
    },

    "getTokenLargestAccounts" : {
        "data" : {
            "jsonrpc": "2.0", "id": 1,
            "method": "getTokenLargestAccounts",
            "params": [
              "{address}"
            ]
          }
    },

    "getTokenSupply" : {
        "data" : {
            "jsonrpc": "2.0",
                "id": 1,
            "method": "getTokenSupply",
            "params": [
              "{address}"
            ]
        }
    }
    
}

In [90]:
import json 

# Task 1: getContractMetadata
getContractMetadata = Task(
    description='Fetch metadata details of the program at the given address. Metadata should include owner, lamports, and any associated tags.',
    expected_output='Contract metadata including owner, lamports, space, and other relevant details.',
    context = [get_task],
    agent=getAgent,  # Replace with the appropriate agent name,
)

# Task 2: getGasPrices
getGasPrices = Task(
    description='Fetch the fees for the message on blockchain.',
    expected_output='All the parameters',
    context = [get_task],
    agent=getAgent,
)

# Task 3: getTransactionDetails
getTransactionDetails = Task(
    description='Fetch the details of the transaction with the given ID. Include sender, receiver, amount, timestamp, status, and confirmation count.',
    expected_output='Transaction details including sender, receiver, amount, timestamp, status (confirmed or pending), and confirmation count.',
    context = [get_task],
    agent=getAgent,
)

# Task 4: getNetworkInfo
getHealth = Task(
    description='Retrieve the details of the blockchain network. Include current block height, number of transactions per second, and network health status.',
    expected_output='Network information including block height, TPS (transactions per second), and network health status (healthy, degraded, etc.).',
    context = [get_task],
    agent=getAgent,
)

# Task 5: getAccountInfo
getAccountInfo = Task(
    description='Fetch the details of the given address on the blockchain and convert the balance into SOL and check if the SOL is greater than 0.',
    expected_output='Balance, address, space ocucpied by the data, and return true is SOL balance is greather than 0 and false otherwise. Give this output in a sentence form and not json.',
    context = [get_task],
    agent=getAgent,
)

# Task 6: requestAirdrop
requestAirdrop = Task(
    description='Request an airdrop of a specified amount of SOL to the given address. Validate if the request was successful and include the updated balance after the airdrop',
    expected_output='Confirmation of airdrop success, transaction ID, and updated balance in SOL.',
    context = [get_task],
    agent=getAgent,
)

# Task 7: getBalance
getBalance = Task(
    description='Get the balance of the given address and convert it to SOL',
    expected_output='Balance in SOL in a sentence form and not json',
    context = [get_task],
    agent=getAgent,
)

# Task 8: getInflationRate
getInflationRate = Task(
    description='Fetch the current inflation rate details of the blockchain, including the total, validator, and foundation inflation rates.',
    expected_output='Inflation rate details including total inflation rate, validator inflation rate, and foundation inflation rate in a sentence format.',
    context=[get_task],
    agent=getAgent,
)

# Task 9: getSupply
getSupply = Task(
    description='Retrieve the total supply of SOL on the blockchain. Include circulating and non-circulating supplies and total.',
    expected_output='Supply details including total supply, circulating supply, and non-circulating supply in a sentence format.',
    context=[get_task],
    agent=getAgent,
)

# Task 10: getTokenAccountBalance
getTokenAccountBalance = Task(
    description='Fetch the balance of the token account for the given address. Include the balance in both the smallest unit (lamports) and human-readable format.',
    expected_output='Token account balance including lamports and equivalent human-readable format, decimals and uiAmountString in sentence format.',
    context=[get_task],
    agent=getAgent
)

# Task 11: getTokenAccountsByDelegate
getTokenAccountsByDelegate = Task(
    description='Retrieve all token accounts delegated to the given delegate address. Include account details and associated balances.',
    expected_output='List of token accounts delegated to the address, including account details, balances and delegated amount.',
    context=[get_task],
    agent=getAgent
)

# Task 12: getTokenAccountsByOwner
getTokenAccountsByOwner = Task(
    description='Retrieve all token accounts owned by the given owner address and given programID. Include account details and associated balances.',
    expected_output='List of token accounts owned by the address, including account details and balances.',
    context=[get_task],
    agent=getAgent
)

# Task 13: getTokenLargestAccounts
getTokenLargestAccounts = Task(
    description='Fetch the largest token accounts for the given token mint. Include the account addresses and their respective balances, decimals and uiAmountString.',
    expected_output='List of the largest token accounts for the token mint, including addresses, balances, decimals and uiAmountString.',
    context=[get_task],
    agent=getAgent
)

# Task 14: getTokenSupply
getTokenSupply = Task(
    description='Retrieve the total supply of a specific token for the given token mint address. Include the total supply in lamports and human-readable format.',
    expected_output='Total token supply for the given mint address in lamports and human-readable format. Also give decimals, uiAmountString',
    context=[get_task],
    agent=getAgent
)



In [87]:
memory = []  # To store multi-turn chat history

In [ ]:
def multi_turn_chat(user_input, crew, memory):
    memory.append({"role": "user", "content": user_input})

    inputs = {"text": user_input, "task_params_map": task_params_map, "memory": memory}
    result = crew.kickoff(inputs)

    memory.append({"role": "assistant", "content": result.raw})
    return result

crew = Crew(
    agents=[task_define],
    tasks=[get_task],
    memory = True,
    verbose=True,
)

result = multi_turn_chat("What is the balance of dv1ZAGvdsz5hHLwWXsVnM94hWf1pjbKVau1QVkaMJ92", crew, memory)
# # result = crew.kickoff(inputs={"text":"Was the SOL greater than 0 in the previous result", "task_params_map": task_params_map})

second_agent_tasks = {
    "getContractMetadata" : getContractMetadata,
    "getGasPrices" : getGasPrices,
    "getTransactionDetails" : getTransactionDetails,
    "getHealth" : getHealth,
    "getAccountInfo" : getAccountInfo,
    "requestAirdrop" : requestAirdrop,
    "getBalance" : getBalance,
    "getInflationRate" : getInflationRate,
    "getSupply" : getSupply,
    "getTokenAccountBalance" : getTokenAccountBalance,
    "getTokenAccountsByDelegate" : getTokenAccountsByDelegate,
    "getTokenAccountsByOwner" : getTokenAccountsByOwner,
    "getTokenLargestAccounts" : getTokenLargestAccounts,
    "getTokenSupply" : getTokenSupply
}

if result['task'] and result['task'] in second_agent_tasks:
    second_task = second_agent_tasks[result['task']]
    crew = Crew(agents=[getAgent], tasks=[second_task], memory=True, verbose=True)
    final_result = crew.kickoff(inputs={"data": result['data']})
    memory.append({"role": "assistant", "content": final_result.raw})

06:12:45 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


# Agent: Task Definer and Populator
## Task: Identify the task needed to be performed in What is the balance of dv1ZAGvdsz5hHLwWXsVnM94hWf1pjbKVau1QVkaMJ92. Return getAccountInfo is the user wants to send SOL to someone.If the user is just conversing and there is no task to be performed, converse with the user like a human and take into consideration [{'role': 'user', 'content': 'Tell me about dv1ZAGvdsz5hHLwWXsVnM94hWf1pjbKVau1QVkaMJ92'}, {'role': 'assistant', 'content': '{\n  "task": "getAccountInfo",\n  "data": {\n    "jsonrpc": "2.0",\n    "id": 1,\n    "method": "getAccountInfo",\n    "params": [\n      "dv1ZAGvdsz5hHLwWXsVnM94hWf1pjbKVau1QVkaMJ92",\n      {\n        "encoding": "base64"\n      }\n    ]\n  }\n}'}, {'role': 'assistant', 'content': 'The address dv1ZAGvdsz5hHLwWXsVnM94hWf1pjbKVau1QVkaMJ92 has a balance of 6736.576564398 SOL, occupies 0 bytes of space, and has a SOL balance greater than 0 (true).\n```'}, {'role': 'user', 'content': 'What is the balance of dv1ZAGvdsz5h

06:12:48 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler




# Agent: Task Definer and Populator
## Final Answer: 
```json
{
  "task": "getBalance",
  "data": {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "getBalance",
    "params": [
      "dv1ZAGvdsz5hHLwWXsVnM94hWf1pjbKVau1QVkaMJ92"
    ]
  }
}
```




06:12:50 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
06:12:51 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


...

06:12:59 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
06:13:00 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini




# Agent: Blockchain Information Retreiver
## Using tool: Wrapper API
## Tool Input: 
"{\"data\": {\"jsonrpc\": \"2.0\", \"id\": 1, \"method\": \"getBalance\", \"params\": [\"dv1ZAGvdsz5hHLwWXsVnM94hWf1pjbKVau1QVkaMJ92\"]}}"
## Tool Output: 
{'jsonrpc': '2.0', 'result': {'context': {'apiVersion': '2.1.8', 'slot': 355798375}, 'value': 6736580226982}, 'id': 1}


06:13:02 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler




# Agent: Blockchain Information Retreiver
## Final Answer: 
The balance of address dv1ZAGvdsz5hHLwWXsVnM94hWf1pjbKVau1QVkaMJ92 is 6736.580226982 SOL.
```




06:13:04 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
06:13:06 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
